In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

import warnings

warnings.filterwarnings("ignore")

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_money = pd.read_csv('train_final.csv')



In [15]:
def reshape_cat_features(data, cast_col, value_col):
        res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
        res.columns.name = None
        res = res.rename(columns={col:cast_col+'_'+str(col) for col in res.columns})
        return res

def data_preprocessing_group(train,test):
    error_data = ['C2085', 'C1397', 'C2431', 'C1649', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']

    for error in error_data :
        train = train[train['단지코드'] != error]


    train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
    test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
    train['임대보증금'] = train['임대보증금'].astype(float)
    test['임대보증금'] = test['임대보증금'].astype(float)

    train.loc[train.임대료=='-', '임대료'] = np.nan
    test.loc[test.임대료=='-', '임대료'] = np.nan
    train['임대료'] = train['임대료'].astype(float)
    test['임대료'] = test['임대료'].astype(float)

    train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
    test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

    cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
    train[cols] = train[cols].fillna(0)
    test[cols] = test[cols].fillna(0)

    test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
    test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

    train = train.drop_duplicates()
    test = test.drop_duplicates()

    unique_cols = ['총세대수', '지역', '공가수', 
                   '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
                   '도보 10분거리 내 버스정류장 수',
                   '단지내주차면수', '등록차량수']
    train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
    test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()



    X_train = pd.concat([train_agg,
                           reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                           reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                           reshape_cat_features(data=train, cast_col='자격유형', value_col='counter'),
                           ], axis=1)

    X_test = pd.concat([test_agg,
                           reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                           reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                           reshape_cat_features(data=test, cast_col='자격유형', value_col='counter'),
                      ], axis=1)
    return X_train,X_test

In [16]:
train.head(1)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0


In [18]:
train_group,test_group = data_preprocessing_group(train,test)
train_group

,총세대수,지역,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,임대건물구분_상가,임대건물구분_아파트,공급유형_공공분양,...,자격유형_F,자격유형_G,자격유형_H,자격유형_I,자격유형_J,자격유형_K,자격유형_L,자격유형_M,자격유형_N,자격유형_O
단지코드,,,,,,,,,,,,,,,,,,,,,
C2515,545,경상남도,17.0,0.0,3.0,624.0,205.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C1407,1216,대전광역시,13.0,1.0,1.0,1285.0,1064.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C1945,755,경기도,6.0,1.0,3.0,734.0,730.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C1470,696,전라북도,14.0,0.0,2.0,645.0,553.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C1898,566,전라북도,9.0,0.0,6.0,517.0,415.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2586,90,제주특별자치도,7.0,0.0,3.0,66.0,57.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
C2035,492,강원도,24.0,0.0,1.0,521.0,246.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C2020,40,부산광역시,7.0,1.0,2.0,25.0,19.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
train_group.columns

Index(['총세대수', '지역', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수', '임대건물구분_상가', '임대건물구분_아파트',
       '공급유형_공공분양', '공급유형_공공임대(10년)', '공급유형_공공임대(50년)', '공급유형_공공임대(5년)',
       '공급유형_공공임대(분납)', '공급유형_국민임대', '공급유형_영구임대', '공급유형_임대상가', '공급유형_장기전세',
       '공급유형_행복주택', '자격유형_A', '자격유형_B', '자격유형_C', '자격유형_D', '자격유형_E', '자격유형_F',
       '자격유형_G', '자격유형_H', '자격유형_I', '자격유형_J', '자격유형_K', '자격유형_L', '자격유형_M',
       '자격유형_N', '자격유형_O'],
      dtype='object')